### Connection & Imports

In [1]:
from sqlalchemy.orm import Session
import sys
sys.path.append('..')
from app import models
from app.database import engine,SessionLocal
from pprint import pprint
import pandas as pd
from fastapi.encoders import jsonable_encoder
import random

### Drop all

In [2]:
models.Base.metadata.drop_all(bind=engine)

### Create all

In [3]:
models.Base.metadata.create_all(bind=engine)

### Table Inspection

In [4]:
from sqlalchemy.engine import reflection
from IPython.display import HTML
insp = reflection.Inspector.from_engine(engine)

for table_name in insp.get_table_names():
    display(HTML(f"<h3>Table: {table_name}</h3>"))
    display(HTML(pd.DataFrame(insp.get_columns(table_name)).to_html(index=False)))

name,type,nullable,default,autoincrement,primary_key
user_id_aktiv,INTEGER,False,None,auto,1
user_id_passiv,INTEGER,False,None,auto,2


name,type,nullable,default,autoincrement,primary_key
dummy_antwort_id,INTEGER,False,None,auto,1
frage_id,INTEGER,True,None,auto,0
antwort,VARCHAR(200),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
frage_id,INTEGER,False,None,auto,1
frage,VARCHAR(200),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
steckbrief_id,INTEGER,False,None,auto,1
freigegeben,BOOLEAN,True,'1',auto,0


name,type,nullable,default,autoincrement,primary_key
steckbrief_id,INTEGER,False,None,auto,1
frage_id,INTEGER,False,None,auto,2
antwort,VARCHAR(200),True,None,auto,0


name,type,nullable,default,autoincrement,primary_key
user_id,INTEGER,False,None,auto,1
abteilung_id,INTEGER,True,None,auto,0
steckbrief_id,INTEGER,True,None,auto,0
beruf,VARCHAR(50),True,None,auto,0
vorname,VARCHAR(50),True,None,auto,0
nachname,VARCHAR(50),True,None,auto,0
geburtsdatum,DATE,True,None,auto,0


In [5]:
df=pd.read_csv("../db/testdata/users.csv",sep=";")
df.to_sql("user",engine, if_exists='append',index=False)

In [6]:
df=pd.read_csv("../db/testdata/steckbriefe.csv",sep=";")
df.to_sql("steckbrief",engine, if_exists='append',index=False)

In [7]:
df=pd.read_csv("../db/testdata/steckbrief_frage.csv",sep=";")
df.to_sql("steckbrief_frage",engine, if_exists='append',index=False)

In [8]:
df=pd.read_csv("../db/testdata/frage.csv",sep=";")
df.to_sql("frage",engine, if_exists='append',index=False)

In [9]:
df=pd.read_csv("../db/testdata/dummy_antwort.csv",sep=";")
df.to_sql("dummy_antwort",engine, if_exists='append',index=False)

In [10]:
df=pd.read_csv("../db/testdata/collection.csv",sep=";")
df.to_sql("collection",engine, if_exists='append',index=False)

In [11]:
pd.read_sql(SessionLocal().query(models.User).statement,SessionLocal().bind).T.to_dict().values()

dict_values([{'user_id': 1, 'abteilung_id': 1, 'steckbrief_id': 1, 'beruf': 'Grafiker', 'vorname': 'Marco', 'nachname': 'Gerken', 'geburtsdatum': datetime.date(1993, 5, 12)}, {'user_id': 2, 'abteilung_id': 1, 'steckbrief_id': 2, 'beruf': 'Softwareentwickler', 'vorname': 'Robert', 'nachname': 'Bertrams', 'geburtsdatum': datetime.date(1993, 5, 12)}, {'user_id': 3, 'abteilung_id': 1, 'steckbrief_id': 3, 'beruf': 'Testerin', 'vorname': 'Mia', 'nachname': 'Lamm', 'geburtsdatum': datetime.date(1993, 5, 12)}, {'user_id': 4, 'abteilung_id': 1, 'steckbrief_id': 4, 'beruf': 'Abteilungsleiterin', 'vorname': 'Leonie', 'nachname': 'Walczyk', 'geburtsdatum': datetime.date(1993, 5, 12)}, {'user_id': 5, 'abteilung_id': 1, 'steckbrief_id': 5, 'beruf': 'Administrator', 'vorname': 'Burkhard', 'nachname': 'Peschek', 'geburtsdatum': datetime.date(1993, 5, 12)}, {'user_id': 6, 'abteilung_id': 1, 'steckbrief_id': 6, 'beruf': 'Prozessentwicklerin', 'vorname': 'Marie', 'nachname': 'Rast', 'geburtsdatum': datet

In [39]:
from sqlalchemy import case

my_case = case([
        (collection.user_id_aktiv != None, 1)],
         else_=0
)

In [45]:
collection = SessionLocal().query(models.Collection).filter(models.Collection.user_id_aktiv==2).subquery()


my_case = case([
        (collection.c.user_id_aktiv != None, 1)],
         else_=0
)

pd.read_sql(SessionLocal().query(models.User,my_case.label('gesammelt')).outerjoin(collection,models.User.user_id==collection.c.user_id_passiv).filter(models.User.user_id!=2).statement, SessionLocal().bind)

,user_id,abteilung_id,steckbrief_id,beruf,vorname,nachname,geburtsdatum,gesammelt
0,1,1,1,Grafiker,Marco,Gerken,1993-05-12,0
1,3,1,3,Testerin,Mia,Lamm,1993-05-12,1
2,4,1,4,Abteilungsleiterin,Leonie,Walczyk,1993-05-12,1
3,5,1,5,Administrator,Burkhard,Peschek,1993-05-12,1
4,6,1,6,Prozessentwicklerin,Marie,Rast,1993-05-12,1
5,7,1,7,Scrum Master,Karl,Paczkowski,1993-05-12,0
6,8,1,8,Product Owner,Fritz,Dogan,1993-05-12,0
7,9,1,9,Product Owner,Edith,Giebel,1993-05-12,0
8,10,1,10,Product Owner,Arthur,Fakher,1993-05-12,0


In [23]:
pd.read_sql(SessionLocal().query(models.Collection.user_id_aktiv,models.Collection.user_id_passiv).filter(models.Collection.user_id_aktiv==2).statement, SessionLocal().bind)

,user_id_aktiv,user_id_passiv
0,2,3
1,2,4
2,2,5
3,2,6


In [49]:
collection = SessionLocal().query(models.Collection).filter(models.Collection.user_id_aktiv==2).subquery()
    
check_collected = case([
    (collection.c.user_id_aktiv != None, 1)],
        else_=0
)

print(SessionLocal().query(models.User,check_collected.label('gesammelt')).outerjoin(collection,models.User.user_id==collection.c.user_id_passiv).filter(models.User.user_id!=2))

SELECT user.user_id AS user_user_id, user.abteilung_id AS user_abteilung_id, user.steckbrief_id AS user_steckbrief_id, user.beruf AS user_beruf, user.vorname AS user_vorname, user.nachname AS user_nachname, user.geburtsdatum AS user_geburtsdatum, CASE WHEN (anon_1.user_id_aktiv IS NOT NULL) THEN ? ELSE ? END AS gesammelt 
FROM user LEFT OUTER JOIN (SELECT collection.user_id_aktiv AS user_id_aktiv, collection.user_id_passiv AS user_id_passiv 
FROM collection 
WHERE collection.user_id_aktiv = ?) AS anon_1 ON user.user_id = anon_1.user_id_passiv 
WHERE user.user_id != ?


In [66]:
for row in SessionLocal().query(models.User,my_case.label('gesammelt')).outerjoin(collection,models.User.user_id==collection.c.user_id_passiv).filter(models.User.user_id!=2).all():
    print(row[0].vorname)

Marco
Mia
Leonie
Burkhard
Marie
Karl
Fritz
Edith
Arthur
